CWPK \#22: Basic Knowledge Graph Management - II
=======================================

Some More Basics About Ontology Components
--------------------------

<div style="float: left; width: 305px; margin-right: 10px;">

<img src="http://kbpedia.org/cwpk-files/cooking-with-kbpedia-305.png" title="Cooking with KBpedia" width="305" />

</div>

We began looking at managing classes in our last installment of the [*Cooking with Python and KBpedia*](https://www.mkbergman.com/cooking-with-python-and-kbpedia/) series. We continue on that quest in this current installment, expanding our basic commands for instances, data and object properties, annotations, property relations, and operators. We do not probe all available methods, but emphasize those most often encountered. See the **Additional Documentation** sources at the end of this installment to get further information.

This present installment, plus the prior one, will complete our initial introduction into the ontology management functions available in [owlready2](http://www.lesfleursdunormal.fr/static/informatique/owlready/index_en.html). From here we next introduce some of the more advanced features in this [API](https://en.wikipedia.org/wiki/Application_programming_interface).

To interact with this installment on [Jupyter Notebook](https://en.wikipedia.org/wiki/Project_Jupyter), we will be using the smaller KKO (Kbpedia Knowledge Ontology), <code>kko.owl</code>, because it is a bit easier to inspect and manipulate.

### General Load Method
Note we are continuing to consolidate our initial load statements into a single routine, which we will often use to begin subsequent installments. We also continue to follow the best practice of using logical names to refer to absolute file addresses.

<div style="background-color:#eee; border:1px dotted #aaa; vertical-align:middle; margin:15px 60px; padding:8px;"><strong>Which environment?</strong> The specific load routine you should choose below depends on whether you are using the online MyBinder service (the 'raw' version) or local files. The example below is based on using local files (though replace with your own local directory specification). If loading from MyBinder, use this <a href="https://raw.githubusercontent.com/Cognonto/CWPK/master/sandbox/builds/ontologies/kko.owl">address for <code>kko.owl</code></a></div>

In [2]:
main = 'C:/1-PythonProjects/kbpedia/sandbox/kko.owl'
skos_file = 'http://www.w3.org/2004/02/skos/core' 

from owlready2 import *
kko = get_ontology(main).load()

skos = get_ontology(skos_file).load()
kko.imported_ontologies.append(skos)

Having executed these methods with the <code>shift+enter</code> or Run command from the main menu, we can now do a quick inspection to see if everything has loaded properly. We again use a listing of subclasses of the <code>Products</code> class:

In [3]:
list(kko.Products.subclasses())

[kko.PrimarySectorProduct,
 kko.SecondarySectorProduct,
 kko.TertiarySectorService]

### Instances
Instance assignments work similarly to classes, except we must relate the new instance to its parent class. In this example, we are adding a <code>camera</code> instance to our <code>SecondarySectorProduct</code> class, itself a subclass of <code>Products</code>. We also add a <code>print</code> statement to confirm our assignment worked OK:

In [5]:
camera = kko.SecondarySectorProduct("camera")
print(camera.name)

camera


Like for classes, there are multiple methods for creating a new instance. Here is another form:

In [48]:
computer = kko.SecondarySectorProduct("computer", 
    namespace = kko)

The <code>.instances()</code> class method can be used to iterate through all Instances of a class (including its subclasses). It returns a generator:

In [49]:
for i in kko.SecondarySectorProduct.instances(): print(i)

kko.camera
kko.computer


Go ahead and substitute '<code>Products</code>' for the class name above to verify it also works on subclasses.

As we learned in the last installment, we can also delete the instance and all of its internal references:

In [43]:
destroy_entity(computer)

<h3 id="Data-and-Object-Properties">Data and Object Properties</h3>
<p>Recall that the basic statement (assertion) in RDF, the base language of our ontologies, is a '<a href="https://en.wikipedia.org/wiki/Semantic_triple" target="_blank" rel="noopener">triple</a>'. The triple has the form of <em>subject - property - object</em> (or <em>s-p-o</em>; also, predicate or relation is a synonym for property). Data and object properties may be inferred over in a <a href="https://en.wikipedia.org/wiki/World_Wide_Web_Consortium" target="_blank" rel="noopener">W3C</a> ontology. Object properties require a defined and named object (instance or class) as its object. Data properties require a datatype as its object, including strings. Annotation properties (see below) also take strings or URIs as their object, but can not be inferred over.</p>
<p>One creates properties in a similar manner to classes. In this example, we are creating the <code>has_color</code> sub-property of the <code>qualities</code> property, itself a sub-property of an <code>ObjectProperty</code>:</p>

In [34]:
with kko:
    class has_color(kko.qualities): pass


You will note in this example that we used the underscore character to separate our names to make the property name more readable. Actually, in KKO (and KBpedia) our preferred convention is to use CamelCase with classes initially capitalized, and properties initially lower case. We do not use the underscore. What is important to note here, however, is that these are only best-practice conventions. There is no absolute requirement for classes, individuals or properties to assume a particular naming form.

To see that our new property has been properly added, we can do our standard list method:

In [ ]:
list(kko.properties())

We can delete properties and all internal references with the <code>destroy_entity</code> method. Further, we can make domain and range assignments and other specifications of properties, some addressed below, the others covered in the **Additional Documentation** section.

### Annotations
As noted, annotations are one of the property types where the objects are names or strings or link addresses (IRIs), but over which no reasoning may occur. Annotations provide the labels, definitions, comments, and pointers to the actual objects in the system. We can assign values in a straightforward manner to annotations (in this case, <code>altLabel</code>):

In [52]:
computer.altLabel = ["Mike's computer","PC", "workstation"]

We can confirm the properties assigned to the <code>computer</code> instance with the <code>get_properties</code> method:

In [53]:
kko.computer.get_properties()

{core.altLabel}

We can also see the actual <code>altLabels</code> were properly assigned:

In [54]:
print(kko.computer.altLabel)

["Mike's computer", 'PC', 'workstation']


You will note that the first label in the example above is shown with double-quotes in order to properly account for the single quote in the label for possession.

Like all properties in the system we can get a listing of the properties and classes that have assigned values for that property with this form of <code>.get_relations</code> method:

In [ ]:
list(skos.altLabel.get_relations())

### Indirect Relations
There is a special method in Owlready2 called '<code>INDIRECT_</code>'. Invoking this brings up results for:
- transitive, symmetric and reflexive properties
- property inheritance (i.e., subproperties)
- classes of an individual
- class inheritance (i.e., parent classes), and
- equivalences (equivalent classes, sameAs individuals).

Here is an example:

In [55]:
print(computer.INDIRECT_altLabel)

["Mike's computer", 'workstation', 'PC']


### Property Restrictions
Owlready2 enables us to also place restrictions on our classes through a special type of class constructed by the system. 

- some - <code>Property.some(Range_Class)</code>
- only - <code>Property.only(Range_Class)</code>
- min - <code>Property.min(cardinality, Range_Class)</code>
- max - <code>Property.max(cardinality, Range_Class)</code>
- exactly - <code>Property.exactly(cardinality, Range_Class)</code>
- value - <code>Property.value(Range_Individual / Literal value)</code>
- has_self - <code>Property.has_self(Boolean value)</code>.

These are the same names and understandings as is used by the Protégé ontology [IDE](https://en.wikipedia.org/wiki/Integrated_development_environment) (see **CWPK #5**). See further **Additional Documentation** for details.

### Operators
Owlready2 provides three logical operators between classes (including class constructs and restrictions):

- ‘&’ - And operator (intersection). For example: Class1 & Class2. It can also be written: And([Class1, Class2])
- ‘|’ - Or operator (union). For example: Class1 | Class2. It can also be written: Or([Class1, Class2])
- Not() - Not operator (negation or complement). For example: Not(Class1).

### Save and Exit

When we are finished with our tests, we can File &rarr; Save and Checkpoint, Rename our output file, or specify it at the command line (substituting your own local preferences):

In [31]:
kko.save(file = "C:/1-PythonProjects/kbpedia/sandbox/kko-test.owl", format = "rdfxml")

### Additional Documentation

Here are links to other owlready2 documentation:
- [instances](https://owlready2.readthedocs.io/en/latest/class.html)
- [properties](https://owlready2.readthedocs.io/en/latest/properties.html)
- [annotations](https://owlready2.readthedocs.io/en/latest/annotations.html)
- [property restrictions and operators](https://owlready2.readthedocs.io/en/latest/restriction.html).


 <div style="background-color:#efefff; border:1px dotted #ceceff; vertical-align:middle; margin:15px 60px; padding:8px;"> 
  <span style="font-weight: bold;">NOTE:</span> This article is part of the <a href="https://www.mkbergman.com/cooking-with-python-and-kbpedia/" style="font-style: italic;">Cooking with Python and KBpedia</a> series. See the <a href="https://www.mkbergman.com/cooking-with-python-and-kbpedia/"><strong>CWPK</strong> listing</a> for other articles in the series. <a href="http://kbpedia.org/">KBpedia</a> has its own Web site.
  </div>

<div style="background-color:#ebf8e2; border:1px dotted #71c837; vertical-align:middle; margin:15px 60px; padding:8px;"> 

<span style="font-weight: bold;">NOTE:</span> This <strong>CWPK 
installment</strong> is available both as an online interactive
file <a href="https://mybinder.org/v2/gh/Cognonto/CWPK/master" ><img src="https://mybinder.org/badge_logo.svg" style="display:inline-block; vertical-align: middle;" /></a> or as a <a href="https://github.com/Cognonto/CWPK" title="CWPK notebook" alt="CWPK notebook">direct download</a> to use locally. Make sure and pick the correct installment number. For the online interactive option, pick the <code>*.ipynb</code> file. It may take a bit of time for the interactive option to load.</div>

<div style="background-color:#feeedc; border:1px dotted #f7941d; vertical-align:middle; margin:15px 60px; padding:8px;"> 
<div style="float: left; margin-right: 5px;"><img src="http://kbpedia.org/cwpk-files/warning.png" title="Caution!" width="32" /></div>I am at best an amateur with Python. There are likely more efficient methods for coding these steps than what I provide. I encourage you to experiment -- which is part of the fun of Python -- and to <a href="mailto:mike@mkbergman.com">notify me</a> should you make improvements.    

</div>